In [10]:
import datetime
import pymongo
import os
from pymongo import MongoClient
from dotenv import load_dotenv
ruta_actual = os.getcwd() 
ruta_padre = os.path.dirname(ruta_actual)
load_dotenv('venv/.env')
import datetime
import random
import json
from bson.objectid import ObjectId

lectura de claves

In [11]:
connection_url_SanManuel =  'mongodb://eefACE:9UzYm7sB5HxR4WqE3TnGpJ2MvDfXcZvA6L8O1K0I7U3N5Y8T4R6W9Q3E6T2O8H7D3E6S4L9I7T8O1L0O@192.168.100.11:27018/' 
ecommerce_main = 'Ecommerce'
eCommerce_inventario = [connection_url_SanManuel, ecommerce_main, 'inventario']
eCommerce_usuarios = [connection_url_SanManuel, ecommerce_main, 'usuario']
eCommerce_solicitudCompra = [connection_url_SanManuel, ecommerce_main, 'solicitudCompra']


funciones

In [12]:
def coneccionDB(mongo_uri:str,database_name:str,collection_name:str):
    '''Permite realizar una coneccion a una colecccion en base de datos'''
    
    try:
        client = MongoClient(mongo_uri)
        db = client[database_name]
        collection = db[collection_name]
        print(f'coneccion exitosa a la coleccion: {database_name}.{collection_name}')
    except Exception as e:
        print(e)
    return collection

## Auxiliar para una simulacion

In [32]:
def randomDates(cantidad):
    fechas = []
    for _ in range(cantidad):
        fecha = datetime.datetime(
            random.randint(2024, 2024),  # Año
            random.randint(1, 6),        # Mes
            random.randint(1, 28),        # Día (asumimos hasta 28 para simplicidad)
            random.randint(8, 20),        # Hora
            random.randint(0, 59),        # Minuto
            random.randint(0, 59)         # Segundo
        )
        fechas.append(fecha)
    fechas.sort()
    return fechas

In [46]:
randomDatesLog = randomDates(10)

### Funciones necesarias

In [47]:
def simularCompra(productos:list):

    for producto in productos:
        cantidad = random.randint(a=1,b=10)
        subtotal = cantidad*producto['precioUnitario']

        producto['cantidad'] = cantidad
        producto['subtotal'] = subtotal
    
    return productos

In [61]:
def crearTicket(insertDate:bool, insertDB:bool):
    """
    Se selecciona un usuario al azar para simular que realizo una compra. Posteriormente se realiza una eleccion aleatoria se una cantidad de elementos, 
    un metodod de pago y se actualiza la informacion relacionada a la compra. 

    
    * insertDate: En caso de ser True, se registrara en fechaPedimiento como la fecha y hora actual. En caso de False se toma de una lista previa
    * insertDB: En caso de ser True, se agrega el registro en base de datos en la coleccion eCommerce_solicitudCompra.

    
    * Return: un diccionario con las siguientes llaves: 
        | Llave             | descripcion |
        |------------------|-------------|
        | idUser           | Selecciona un usuario aleatorio de la coleccion usuario.usuario       |
        | userName         | De ese registro, se obtiene el atributo usuario.nombre                |
        | numeroSolicitud  | Se consultala cantidad de registros en la coleccion de solicitudCompra y se le suma 1|
        | compras          | Una lista de diccionarios que tienen las siguientes llaves: ['codigoBarras', 'nombre', 'precioUnitario', 'cantidad', 'subtotal']|
        | totalCompra      | La suma de los subtotal de cada producto |
        | totalArticulos   | La suma de los cantidad de cada producto|
        | metidoPago       | Se simula entre ['Efectivo', 'PayPal', 'Tarjeta de credito', 'Tarjeta de debito']|
        | fechaPedimiento' | En caso de ser True, se registrara en fechaPedimiento como la fecha y hora actual. En caso de False se toma de una lista previa|
        |
    Los valores en los diccionarios en la lista de compras provienen de una consulata a la coleccion inventario
    """

    registro = dict()

    usuario = coneccionDB(*eCommerce_usuarios).find_one({'nombre': "Daniel García"})
    registro['idUser'] = usuario['usuario']
    registro['userName'] = usuario['nombre']
    registro['numeroSolicitud'] = coneccionDB(*eCommerce_solicitudCompra).count_documents({})+1
    #
    sale_size = random.randint(a=1, b= 10)
    pipeline = [{'$sample': {'size': sale_size}}]
    productos  = list(coneccionDB(*eCommerce_inventario).aggregate(pipeline))

    firstkeys = ['codigoBarras', 'nombre', 'precioUnitario']
    productos = list(map(lambda d: {k: d[k] for k in firstkeys}, productos))
    productos = simularCompra(productos=productos)
    
    registro['compras'] = productos
    #
    registro['totalCompra']    =  sum([producto['subtotal'] for producto in productos])
    registro['totalArticulos'] =  sum([producto['cantidad'] for producto in productos])
    registro['metidoPago']     =  random.choice(['Efectivo', 'PayPal', 'Tarjeta de credito', 'Tarjeta de debito'])

    if insertDate:
        registro['fechaPedimiento'] = datetime.datetime.now() 
    else:
        registro['fechaPedimiento']= randomDatesLog.pop(0)
    if insertDB:
        coneccionDB(*eCommerce_solicitudCompra).insert_one(registro)
    return registro

In [49]:
def actualizarRegistros(productos:list):
    eCommerceInventarioDB = coneccionDB(*eCommerce_inventario)
    for producto in productos:
            
            producto_bd = list(eCommerceInventarioDB.find({"nombre":{"$in":[producto['nombre']]}}))
            # filter = {nombre}
            if len(producto_bd) > 0:
                
                try:
                    cantidad_inventario = producto_bd[0]['Total']
                    nuevo_total = cantidad_inventario - producto['cantidad']
                    name = producto['nombre']

                    if nuevo_total < 0:
                        print("No existe una cantidad en almacen para organizar esa demanda")
                    elif nuevo_total == 0:
                        print("ya no queda mas en inventario")
                    else:
                        print("Ahora solo quedan {} unidades del producto {}".format(nuevo_total, name))

                    if nuevo_total >=0:
                        filter = {'nombre': name}
                        actualizacion = {"$set": {"Total": nuevo_total}}
                        resultado = eCommerceInventarioDB.update_one(filter, actualizacion)
                    else:
                         print("no se actualiza este cambio")
                
                except:
                    None

In [76]:
ticket = crearTicket(insertDate=False, insertDB=False)
actualizarRegistros(productos=ticket['compras'])

coneccion exitosa a la coleccion: Ecommerce.usuario
coneccion exitosa a la coleccion: Ecommerce.solicitudCompra
coneccion exitosa a la coleccion: Ecommerce.inventario
coneccion exitosa a la coleccion: Ecommerce.inventario
Ahora solo quedan 55 unidades del producto Puños en los tobillos.
Ahora solo quedan 43 unidades del producto SEDUCTOR - Lujuria doblada
Ahora solo quedan 47 unidades del producto Secreto privado- Varita de alegría
Ahora solo quedan 90 unidades del producto Kits de Bondage-Línea Leopardo Rojo
Ahora solo quedan 291 unidades del producto Anillo para el pene pesado - Estimulador
Ahora solo quedan 41 unidades del producto SUCUBIS DE MODA - DONG VIBRANTE
Ahora solo quedan 42 unidades del producto AMBIENTE ENCANTADOR - VARITA MÁGICA
Ahora solo quedan 54 unidades del producto Bola Banger para pene -Ring 2 bolas


In [75]:
# ticket = crearTicket(insertDate=False, insertDB=False)
print(ticket['compras'][0]['nombre'])

eCommerceInventarioDB = coneccionDB(*eCommerce_inventario)

name = ticket['compras'][0]['nombre']
nuevo_total = 50


filter = {'nombre': name}

actualizacion = {"$set": {"Total": nuevo_total}}

resultado = eCommerceInventarioDB.update_one(filter, actualizacion)

PENETRADOR - Pene realista
coneccion exitosa a la coleccion: Ecommerce.inventario


'Pene de bombeo'

In [62]:
ticket = crearTicket(insertDate=False, insertDB=True)
# actualizarRegistros(productos=ticket['compras'])
 ticket['compras'][0]['nombre']

coneccion exitosa a la coleccion: Ecommerce.usuario
coneccion exitosa a la coleccion: Ecommerce.solicitudCompra
coneccion exitosa a la coleccion: Ecommerce.inventario
coneccion exitosa a la coleccion: Ecommerce.solicitudCompra


In [66]:
# ticket['compras'][0].keys()

'Buscador implacable - Huevo remoto de silicona'

In [24]:


fechas = randomDates(10)
fechas[0]

datetime.datetime(2024, 1, 1, 14, 34, 24)

In [26]:
fechas[0] < datetime.datetime.now()

True

ejemplo de varias compras